In [1]:
import os

In [2]:
%pwd

'd:\\MEDICINAL_PLANT_FINAL\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MEDICINAL_PLANT_FINAL'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from src.Medicinal_Plant_Identification.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.Medicinal_Plant_Identification.utils.common import read_yaml,create_directories
import tensorflow as tf


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Medicinal plant dataset")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            initial_epoch=50,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-07-16 21:49:20,432: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-07-16 21:49:20,461: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-16 21:49:20,466: INFO: common: created directory at: artifacts]
[2024-07-16 21:49:20,469: INFO: common: created directory at: artifacts\training]
Found 564 images belonging to 40 classes.
Found 2381 images belonging to 40 classes.
Epoch 51/60
119/119 [==============================] - 712s 6s/step - loss: 0.5931 - accuracy: 0.8314 - val_loss: 0.8034 - val_accuracy: 0.7875
Epoch 52/60
119/119 [==============================] - 588s 5s/step - loss: 1.3022 - accuracy: 0.7827 - val_loss: 0.9828 - val_accuracy: 0.7768
Epoch 53/60
119/119 [==============================] - 675s 6s/step - loss: 0.6545 - accuracy: 0.8149 - val_loss: 0.6959 - val_accuracy: 0.8143
Epoch 54/60
119/119 [==============================] - 558s 5s/step - loss: 0.9702 - accuracy: 0.7946 - val_loss: 0.5467 - val_accuracy: 0.8196
Epoch 55/60
119/119 [==============================] - 602s 5s/step - loss: 0.7030 - accuracy: 0.795

C:\Users\High-Tech\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
model.save('model_epoch_60.h5')


NameError: name 'model' is not defined

In [29]:
# # model2 = tf.keras.models.load_model("artifacts/training/model_epoch_50.h5")
# training_data_path= os.path.join("artifacts/data_ingestion", "Medicinal plant dataset")
# training_data_path
# import tensorflow as tf

# # Directory where the data is located
# data_dir = training_data_path

# # Parameters
# batch_size = 16
# img_height = 224
# img_width = 224
# validation_split = 0.2



# # Load training data
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     data_dir,
#     validation_split=validation_split,
#     subset="training",
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size
# )

# # Load validation data
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     data_dir,
#     validation_split=validation_split,
#     subset="validation",
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size
# )
# train_ds.class_names
# len(train_ds)
